In [162]:
import requests
from bs4 import BeautifulSoup

In [47]:
# getting the website url
url = 'https://www.latam.com/es_mx/apps/personas/booking?fecha1_dia=24&fecha1_anomes=2021-11&auAvailability=1&ida_vuelta=ida&vuelos_origen=Canc%C3%BAn&from_city1=CUN&vuelos_destino=Nueva%20York&to_city1=LIM&flex=1&vuelos_fecha_salida_ddmmaaaa=26/09/2021&cabina=Y&nadults=1&nchildren=0&ninfants=0&cod_promo=&stopover_outbound_days=0&stopover_inbound_days=0&application=#/'

In [48]:
agent = {"User-Agent":"Mozilla/5.0"}
r = requests.get(url, headers=agent)

In [49]:
# verifying that everything went well
r.status_code

200

In [50]:
s = BeautifulSoup(r.text, 'lxml')

In [52]:
from selenium import webdriver

In [53]:
# I opened the driver in a private browsing window
options = webdriver.ChromeOptions()
options.add_argument('--incognito')
driver = webdriver.Chrome(executable_path='C:/chromedriver/chromedriver.exe', options=options )

In [54]:
# I brought the site url
driver.get(url)

In [56]:
# This xpath shows me all the flights
vuelos = driver.find_elements_by_xpath('//li[@class="flight"]')
vuelos

[<selenium.webdriver.remote.webelement.WebElement (session="d5f799f2350e1311b47c0ce022e8bfa7", element="60d65d0b-745f-401b-9b7a-76e03010f127")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d5f799f2350e1311b47c0ce022e8bfa7", element="ab9e2521-08bb-4cd3-a76b-5e4c592d03a3")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d5f799f2350e1311b47c0ce022e8bfa7", element="d65196d5-c3ec-453e-9081-81cebc13c9f5")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d5f799f2350e1311b47c0ce022e8bfa7", element="21e0efe5-3b4f-45e7-b424-599aa067be77")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d5f799f2350e1311b47c0ce022e8bfa7", element="15009e12-7526-4532-b000-8dcf47504c5c")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d5f799f2350e1311b47c0ce022e8bfa7", element="9a7777e7-0bf6-4edb-af7a-b3d7627ea72e")>]

In [57]:
# I selected the first flight
vuelo = vuelos[0]
vuelo

<selenium.webdriver.remote.webelement.WebElement (session="d5f799f2350e1311b47c0ce022e8bfa7", element="60d65d0b-745f-401b-9b7a-76e03010f127")>

In [58]:
vuelo.find_element_by_xpath('.//div[@class="departure"]//abbr').text

'CUN'

In [59]:
vuelo.find_element_by_xpath('.//div[@class="arrival"]//abbr').text

'LIM'

In [60]:
# Getting departure time
vuelo.find_element_by_xpath('.//div[@class="departure"]/time').get_attribute('datetime')

'16:15'

In [61]:
#Getting the arrival time
vuelo.find_element_by_xpath('.//div[@class="arrival"]/time').get_attribute('datetime')

'21:25'

In [62]:
#Getting the flight duration
vuelo.find_element_by_xpath('.//span[@class="duration"]/time').get_attribute('datetime')

'PT5H10M'

In [63]:
scales_button = vuelo.find_element_by_xpath('.//div[@class="flight-summary-stops-description"]/button')
scales_button

<selenium.webdriver.remote.webelement.WebElement (session="d5f799f2350e1311b47c0ce022e8bfa7", element="681df1c4-4a8b-46bb-a427-2cfe87f21210")>

In [64]:
#We click on the button to access the scales
scales_button.click()

In [65]:
segments = vuelo.find_elements_by_xpath('//div[@class="modal-body sc-bZQynM bTTDW"]/div[@class="sc-hZSUBg gfeULV"]')
segments

[<selenium.webdriver.remote.webelement.WebElement (session="d5f799f2350e1311b47c0ce022e8bfa7", element="caf63b68-6fc8-4cb7-9f96-532389547218")>]

In [66]:
#The scale page is divided by segments, we can know the number of scales by the number of segments - 1
scales = len(segments) - 1
scales

0

In [67]:
segment = segments[0]

In [71]:
#I close the modal
driver.find_element_by_xpath('//div[@class="modal-header sc-dnqmqq cGfTsx"]//button[@class="close"]').click()

In [72]:
#I click on the flight to see the rates
vuelo.click()

In [73]:
#I bring the 3 different rates
rates = vuelo.find_elements_by_xpath('.//div[@class="fares-table-container"]//tfoot//td[contains(@class, "fare-")]')
rates

[<selenium.webdriver.remote.webelement.WebElement (session="d5f799f2350e1311b47c0ce022e8bfa7", element="db887ede-12cf-44fe-b197-5be94b4aae3b")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d5f799f2350e1311b47c0ce022e8bfa7", element="0e2adf15-9a59-4f36-8a75-d36f279bc54f")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d5f799f2350e1311b47c0ce022e8bfa7", element="0106e327-c348-46eb-92e7-dee18b1c21bb")>]

In [143]:
# This function returns a list of dictionaries with the different rates
def get_prices(vuelo):
    rates = vuelo.find_elements_by_xpath('.//div[@class="fares-table-container"]//tfoot//td[contains(@class, "fare-")]')
    prices = []
    for rate in rates:
        name = rate.find_element_by_xpath('.//label').get_attribute('for')
        currency = rate.find_element_by_xpath('.//span[@class="price"]/span[@class="currency-symbol"]').text
        value = rate.find_element_by_xpath('.//span[@class="price"]/span[@class="value"]').text
        dict_rate = {name:{'currency':currency, 'value':value}}
        prices.append(dict_rate)
    return prices

In [148]:
def get_flight_data(vuelo):
    summary = vuelo.find_elements_by_xpath('.//div[@class="flight-summary with-duration"]')
    info_scales = []
    for i in summary:
        origin = vuelo.find_element_by_xpath('.//div[@class="departure"]//abbr').text
        dep_time = vuelo.find_element_by_xpath('.//div[@class="departure"]/time').get_attribute('datetime')
        dest = vuelo.find_element_by_xpath('.//div[@class="arrival"]//abbr').text
        arr_time = vuelo.find_element_by_xpath('.//div[@class="arrival"]/time').get_attribute('datetime')
        flight_time = vuelo.find_element_by_xpath('.//span[@class="duration"]/time').get_attribute('datetime')
        data_dic = {
            'origin':origin,
            'dep_time':dep_time,
            'dest':dest,
            'arr_time':arr_time,
            'flight_time':flight_time,
        }
        info_scales.append(data_dic)
    return info_scales
    

In [145]:
def get_info(vuelo):
    vuelos = driver.find_elements_by_xpath('//li[@class="flight"]')
    print(f'There are {len(vuelos)} flights.')
    print('Scraping...')
    info = []
    for vuelo in vuelos:
        # I wet the data of each flight
        flight_data = get_flight_data(vuelo)
        vuelo.click()
        prices = get_prices(vuelo)
        vuelo.click()
        info.append({'prices':prices, 'flight_data':flight_data})
    return info

In [161]:
import time

In [163]:
# I opened the driver in a private browsing window
options = webdriver.ChromeOptions()
options.add_argument('--incognito')
driver = webdriver.Chrome(executable_path='C:/chromedriver/chromedriver.exe', options=options )
driver.get(url)
# I need to introduce a delay
time.sleep(16)
get_info(driver)

There are 6 flights.
Scraping...


[{'prices': [{'LIGHT': {'currency': 'US$', 'value': '554'}},
   {'PLUS': {'currency': 'US$', 'value': '644'}},
   {'TOP': {'currency': 'US$', 'value': '751'}}],
  'flight_data': [{'origin': 'CUN',
    'dep_time': '16:15',
    'dest': 'LIM',
    'arr_time': '21:25',
    'flight_time': 'PT5H10M'}]},
 {'prices': [{'LIGHT': {'currency': 'US$', 'value': '554'}},
   {'PLUS': {'currency': 'US$', 'value': '644'}},
   {'TOP': {'currency': 'US$', 'value': '751'}}],
  'flight_data': [{'origin': 'CUN',
    'dep_time': '18:20',
    'dest': 'LIM',
    'arr_time': '23:30',
    'flight_time': 'PT5H10M'}]},
 {'prices': [{'LIGHT': {'currency': 'US$', 'value': '266'}},
   {'PLUS': {'currency': 'US$', 'value': '309'}},
   {'TOP': {'currency': 'US$', 'value': '360'}}],
  'flight_data': [{'origin': 'CUN',
    'dep_time': '12:21',
    'dest': 'LIM',
    'arr_time': '22:25',
    'flight_time': 'PT10H4M'}]},
 {'prices': [{'LIGHT': {'currency': 'US$', 'value': '266'}},
   {'PLUS': {'currency': 'US$', 'value': '

In [164]:
driver.close()